### Import libraries

In [1]:
# !pip install python-chess
# !pip install python-chess --upgrade 
from stockfish import Stockfish
from stockfish import StockfishException
import pandas as pd
import numpy as np
import re
import chess
import chess.engine
import random
import concurrent.futures
# from functools import partial

### Import dataset

In [2]:
with open('posiciones.txt') as f:
    lines = f.readlines()
pos = []
for i in range(len(lines)):
    pos.append(re.sub('\n','', lines[i]))    
    

We duplicate each position: In the first one, black makes the next move and in the second one, white makes it 

In [3]:
bl = []
wh = []
for i in range(len(pos)):
    bl.append(pos[i] + ' b')
    wh.append(pos[i] + ' w')
dataset = bl + wh

### Additional functions

In [4]:
def evaluationND(board, time_limit, rep):
    try:
        values = []
        types = []
        engine = chess.engine.SimpleEngine.popen_uci("Gull 3 x64.exe") # adding the engine
        for _ in range(rep):
            result = engine.analyse(chess.Board(board), chess.engine.Limit(time=time_limit)) #adding the board and the time limit
            score = str(result['score']) #transforming in string the result output
            string = score[9:] #selecting the needed characters
            pattern = re.compile(r'([^(]+)') #applying regular expressions to select the type
            value = int(string[string.find('(')+1:string.find(')')]) #selecting the value

            values.append(value if board[-1] == 'w' else -value) #adding the value depending on the next move colour
            types.append(re.search(pattern, string).group(1)) #adding the type
        engine.quit()
        return{"type":types, "value":values}
    except:
        pass

In [5]:
def next_board(pos, mov):
    def Convert(string):
        list1 = []
        list1[:0] = string
        return list1
    pos_transformed = []
    colour = pos[-1]
    aux=pos.replace('8','11111111').replace('7','1111111').replace('6','111111').replace('5','11111').replace('4','1111').replace('3','111').replace('2','11')        
    aux2=aux.split('/')
    aux2[7] = aux2[7][:8]

    for i in range(8):
        aux2[i] = Convert(aux2[i])
    aux2[8-int(mov[3])][ord(mov[2]) - ord('a')] = aux2[8-int(mov[1])][ord(mov[0]) - ord('a')]
    aux2[8-int(mov[1])][ord(mov[0]) - ord('a')] = '1'
    aux3 = [] * 8
    for i in range(8):
        aux3.append([])
        aux3[i] = "".join(aux2[i])
    pos_transformed = "/".join(aux3)
   
    if colour == 'b':
        pos_transformed = pos_transformed + ' w'
    else:
        pos_transformed = pos_transformed + ' b'
    pos_transformed = pos_transformed.replace('11111111','8').replace('1111111','7').replace('111111','6').replace('11111','5').replace('1111','4').replace('111','3').replace('11','2')        

    return pos_transformed

In [6]:
def next_move(board, time_limit):
    try:
        engine = chess.engine.SimpleEngine.popen_uci("Houdini_15a_x64.exe") # adding the engine
        result = engine.analyse(chess.Board(board), chess.engine.Limit(time=time_limit), multipv=2)
        return str(result[0].get('pv')[0])
    except:
        pass

In [7]:
next_board(dataset[0], next_move(dataset[0],1))

'1b1B1Qr1/7p/6r1/2P5/4k3/1K6/4B3/8 w'

In [8]:
def dif1(val1,val2):
    if abs(val1)+abs(val2) == 0:
        return 0
    else:
        return abs(val1-val2)/(abs(val1)+abs(val2))
def dif2(val1,val2):
    return float(abs(val1-val2))

### Followup inputs generation

In [9]:
def sim_axis(board):
    colour = board[-1]
    pos = board[:-2]
    rows = pos.split("/")
    new_rows = []
    for j in range(8):
        new_rows.append(rows[j][::-1])
    return '/'.join(new_rows) + ' ' + colour

In [10]:
def sim_diag(pos):
    def Convert(string):
        list1 = []
        list1[:0] = string
        return list1
    pos_transformed = []
    colour = pos[-1:]
    aux=pos.replace('8','11111111').replace('7','1111111').replace('6','111111').replace('5','11111').replace('4','1111').replace('3','111').replace('2','11')         
    aux2=aux.split('/')
    for i in range(8):
        aux2[i] = Convert(aux2[i])
        aux3 = [] * 8

    for j in reversed(range(8)):
        aux4=[]
        for i in reversed(range(8)):
            aux4.append(aux2[i][j])
        aux3.append(aux4)
    for i in reversed(range(8)):
        aux3[i] = "".join(aux3[i])
    pos_transformed = "/".join(aux3)
    if colour == 'b':
        pos_transformed = pos_transformed + ' b'
    else:
        pos_transformed = pos_transformed + ' w'
    pos_transformed = pos_transformed.replace('11111111','8').replace('1111111','7').replace('111111','6').replace('11111','5').replace('1111','4').replace('111','3').replace('11','2')         

    return pos_transformed

def has_pawns(pos):
    return pos.find('p') != -1 or pos.find('P') != -1

In [11]:
def next_board(pos, mov):
    def Convert(string):
        list1 = []
        list1[:0] = string
        return list1
    pos_transformed = []
    colour = pos[-1]
    aux=pos.replace('8','11111111').replace('7','1111111').replace('6','111111').replace('5','11111').replace('4','1111').replace('3','111').replace('2','11')        
    aux2=aux.split('/')
    aux2[7] = aux2[7][:8]

    for i in range(8):
        aux2[i] = Convert(aux2[i])
    aux2[8-int(mov[3])][ord(mov[2]) - ord('a')] = aux2[8-int(mov[1])][ord(mov[0]) - ord('a')]
    aux2[8-int(mov[1])][ord(mov[0]) - ord('a')] = '1'
    aux3 = [] * 8
    for i in range(8):
        aux3.append([])
        aux3[i] = "".join(aux2[i])
    pos_transformed = "/".join(aux3)
   
    if colour == 'b':
        pos_transformed = pos_transformed + ' w'
    else:
        pos_transformed = pos_transformed + ' b'
    pos_transformed = pos_transformed.replace('11111111','8').replace('1111111','7').replace('111111','6').replace('11111','5').replace('1111','4').replace('111','3').replace('11','2')        

    return pos_transformed

In [12]:
def replace(pos):
    aux = pos
    if pos.count('B') > 0 & pos.count('R') > 0:
        n = random.random()
        if n > 0.5:
            aux = pos.replace('R','Q',1)
        else:
            aux=pos.replace('B','Q',1)
    elif pos.count('B') > 0 & pos.count('R') == 0:
        aux = pos.replace('B','Q',1)
    elif pos.count('R') > 0 & pos.count('B') == 0:
        aux = pos.replace('R','Q',1)
    return aux


In [13]:
def sim_mirror(board):    
    colour = board[-1:]
    pos = board[:-2]
    rows = pos.swapcase().split("/")
    new_rows = []
    for j in reversed(rows):
        new_rows.append(j)
    new_board = '/'.join(new_rows)
    new_board += ' w' if colour == 'b' else ' b'
    return new_board

In [14]:
def checkcheckW(pos):
    def make_matrix(board): #type(board) == chess.Board()
        pgn = board
        foo = []  #Final board
        pieces = pgn.split(" ", 1)[0]
        rows = pieces.split("/")
        for row in rows:
            foo2 = []  #This is the row I make
            for thing in row:
                if thing.isdigit():
                    for i in range(0, int(thing)):
                        foo2.append('.')
                else:
                    foo2.append(thing)
            foo.append(foo2)
        return foo
    whites = ['Q','K','R','B','N','P']
    blacks = ['q', 'k', 'r', 'b', 'n', 'p']
    a = make_matrix(pos)
    check = bool()
    pos = []
    for i in range(8):
        for j in range(8):
            if a[i][j] == 'K':
                pos.append(i)
                pos.append(j)
    for i in list(reversed(range(pos[1]))):
        if a[pos[0]][i] in whites or a[pos[0]][i] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'q' or a[pos[0]][i]== 'r':
            check = True
            break
    for i in list(range(pos[1]+1,8)):
        if a[pos[0]][i] in whites or a[pos[0]][i] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'q' or a[pos[0]][i]== 'r':
            check = True
            break
    for i in list(reversed(range(pos[0]))):
        if a[i][pos[1]] in whites or a[i][pos[1]] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'q' or a[i][pos[1]]== 'r':
            check = True
            break
    for i in list(range(pos[0]+1,8)):
        if a[i][pos[1]] in whites or a[i][pos[1]] in list(blacks[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'q' or a[i][pos[1]]== 'r':
            check = True
            break
    top_right = np.minimum(abs(pos[0]) + 1, abs(8 - pos[1]))
    top_left = np.minimum(abs(pos[0]) + 1, abs(pos[1]) + 1)
    bot_right = np.minimum(abs(8 - pos[0]), abs(8 - pos[1]))
    bot_left = np.minimum(abs(8 - pos[0]), abs(pos[1]) + 1)
    for i in list(range(1,bot_right)):
        if a[pos[0]+i][pos[1]+i] in whites or a[pos[0]+i][pos[1]+i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]+i] == 'b' or a[pos[0]+i][pos[1]+i] == 'q':
            check = True
            break
    for i in list(range(1,top_right)):
        if a[pos[0]-i][pos[1]+i] in whites or a[pos[0]-i][pos[1]+i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]+i] == 'b' or a[pos[0]-i][pos[1]+i] == 'q':
            check = True
            break
    for i in list(range(1,top_left)):
        if a[pos[0]-i][pos[1]-i] in whites or a[pos[0]-i][pos[1]-i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]-i] == 'b' or a[pos[0]-i][pos[1]-i] == 'q':
            check = True
            break
    for i in list(range(1,bot_left)):
        if a[pos[0]+i][pos[1]-i] in whites or a[pos[0]+i][pos[1]-i] in list(blacks[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]-i] == 'b' or a[pos[0]+i][pos[1]-i] == 'q':
            check = True
            break
    if pos[0]+2 < 8 and pos[1]-1 >= 0 and a[pos[0]+2][pos[1]-1] == 'n':
        check = True
    if pos[0]+2 < 8 and pos[1]+1 < 8 and a[pos[0]+2][pos[1]+1] == 'n':
        check = True
    if pos[0]-2 >= 0 and pos[1]-1 >= 0 and a[pos[0]-2][pos[1]-1] == 'n':
        check = True
    if pos[0]-2 >= 0 and pos[1]+1 < 8 and a[pos[0]-2][pos[1]+1] == 'n':
        check = True
    if pos[0]+1 < 8 and pos[1]-2 >= 0 and a[pos[0]+1][pos[1]-2] == 'n':
        check = True
    if pos[0]+1 < 8 and pos[1]+2 < 8 and a[pos[0]+1][pos[1]+2] == 'n':
        check = True
    if pos[0]-1 >= 0 and pos[1]-2 >= 0 and a[pos[0]-1][pos[1]-2] == 'n':
        check = True
    if pos[0]-1 >= 0 and pos[1]+2 < 8 and a[pos[0]-1][pos[1]+2] == 'n':
        check = True
    if pos[0]-1 >= 0 and pos[1]-1 >= 0 and a[pos[0]-1][pos[1]-1] == 'p':
        check = True
    if pos[0]-1 >= 0 and pos[1]+1 < 8 and a[pos[0]-1][pos[1]+1] == 'p':
        check = True
    return check

In [15]:
def checkcheckB(pos):
    def make_matrix(board): #type(board) == chess.Board()
        pgn = board
        foo = []  #Final board
        pieces = pgn.split(" ", 1)[0]
        rows = pieces.split("/")
        for row in rows:
            foo2 = []  #This is the row I make
            for thing in row:
                if thing.isdigit():
                    for i in range(0, int(thing)):
                        foo2.append('.')
                else:
                    foo2.append(thing)
            foo.append(foo2)
        return foo
    whites = ['Q','K','R','B','N','P']
    blacks = ['q', 'k', 'r', 'b', 'n', 'p']
    a = make_matrix(pos)
    check = bool()
    pos = []
    for i in range(8):
        for j in range(8):
            if a[i][j] == 'k':
                pos.append(i)
                pos.append(j)
    for i in list(reversed(range(pos[1]))):
        if a[pos[0]][i] in blacks or a[pos[0]][i] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'Q' or a[pos[0]][i]== 'R':
            check = True
            break
    for i in list(range(pos[1]+1,8)):
        if a[pos[0]][i] in blacks or a[pos[0]][i] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[pos[0]][i] == 'Q' or a[pos[0]][i]== 'R':
            check = True
            break
    for i in list(reversed(range(pos[0]))):
        if a[i][pos[1]] in blacks or a[i][pos[1]] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'Q' or a[i][pos[1]]== 'R':
            check = True
            break
    for i in list(range(pos[0]+1,8)):
        if a[i][pos[1]] in blacks or a[i][pos[1]] in list(whites[j] for j in [1,3,4,5]):
            break
        elif a[i][pos[1]] == 'Q' or a[i][pos[1]]== 'R':
            check = True
            break
    top_right = np.minimum(abs(pos[0]) + 1, abs(8 - pos[1]))
    top_left = np.minimum(abs(pos[0]) + 1, abs(pos[1]) + 1)
    bot_right = np.minimum(abs(8 - pos[0]), abs(8 - pos[1]))
    bot_left = np.minimum(abs(8 - pos[0]), abs(pos[1]) + 1)
    for i in list(range(1,bot_right)):
        if a[pos[0]+i][pos[1]+i] in blacks or a[pos[0]+i][pos[1]+i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]+i] == 'B' or a[pos[0]+i][pos[1]+i] == 'Q':
            check = True
            break
    for i in list(range(1,top_right)):
        if a[pos[0]-i][pos[1]+i] in blacks or a[pos[0]-i][pos[1]+i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]+i] == 'B' or a[pos[0]-i][pos[1]+i] == 'Q':
            check = True
            break
    for i in list(range(1,top_left)):
        if a[pos[0]-i][pos[1]-i] in blacks or a[pos[0]-i][pos[1]-i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]-i][pos[1]-i] == 'B' or a[pos[0]-i][pos[1]-i] == 'Q':
            check = True
            break
    for i in list(range(1,bot_left)):
        if a[pos[0]+i][pos[1]-i] in blacks or a[pos[0]+i][pos[1]-i] in list(whites[j] for j in [1,2,4,5]):
            break
        elif a[pos[0]+i][pos[1]-i] == 'B' or a[pos[0]+i][pos[1]-i] == 'Q':
            check = True
            break
    if pos[0]+2 < 8 and pos[1]-1 >= 0 and a[pos[0]+2][pos[1]-1] == 'N':
        check = True
    if pos[0]+2 < 8 and pos[1]+1 < 8 and a[pos[0]+2][pos[1]+1] == 'N':
        check = True
    if pos[0]-2 >= 0 and pos[1]-1 >= 0 and a[pos[0]-2][pos[1]-1] == 'N':
        check = True
    if pos[0]-2 >= 0 and pos[1]+1 < 8 and a[pos[0]-2][pos[1]+1] == 'N':
        check = True
    if pos[0]+1 < 8 and pos[1]-2 >= 0 and a[pos[0]+1][pos[1]-2] == 'N':
        check = True
    if pos[0]+1 < 8 and pos[1]+2 < 8 and a[pos[0]+1][pos[1]+2] == 'N':
        check = True
    if pos[0]-1 >= 0 and pos[1]-2 >= 0 and a[pos[0]-1][pos[1]-2] == 'N':
        check = True
    if pos[0]-1 >= 0 and pos[1]+2 < 8 and a[pos[0]-1][pos[1]+2] == 'N':
        check = True
    if pos[0]-1 >= 0 and pos[1]-1 >= 0 and a[pos[0]-1][pos[1]-1] == 'P':
        check = True
    if pos[0]-1 >= 0 and pos[1]+1 < 8 and a[pos[0]-1][pos[1]+1] == 'P':
        check = True
    return check

In [16]:
def bothcheck(pos):
    return checkcheckB(pos) and checkcheckW(pos)

def checkMover(pos):
    return checkcheckB(pos) and pos[-1] == 'w' or checkcheckW(pos) and pos[-1] == 'b'

### Metamorphic Relations

In [17]:
def MR_equi(o1,o2,delta,epsilon):
    final = bool()
    if not( all(elemento == o1.get('type')[0] for elemento in o1.get('type')) or all(elemento == o2.get('type')[0] for elemento in o2.get('type'))):
        final = False
    elif o1.get('type')!= o2.get('type'):
        final = False
    elif o1.get('type') == o2.get('type') and o1.get('type')[0] == 'Mate' and o1.get('value') != o2.get('value'):
        final = False
    elif o1.get('type') == o2.get('type') and o1.get('type')[0] == 'Cp' and o1.get('type')[0] == 'Cp' and dif1(np.mean(o1.get('value')),np.mean(o2.get('value'))) > delta and dif2(np.mean(o1.get('value')),np.mean(o2.get('value'))) >= epsilon:
        final = False
    else:
        final = True
    return final

In [18]:
def MR_better(o1,o2,delta,epsilon):
    final = bool()
    if not(all(elemento == o1.get('type')[0] for elemento in o1.get('type'))) or not(all(elemento == o2.get('type')[0] for elemento in o2.get('type'))):
        final = False
    if o1.get('type') != o2.get('type') and o2.get('type')[0] != 'Mate':
        final = False
    elif o1.get('type') == o2.get('type') == 'Mate' and np.mean(o1.get('value')) < np.mean(o2.get('value')) and np.mean(o1.get('value')) > 0:
        final = False
    elif o1.get('type') == o2.get('type') == 'Mate' and np.mean(o1.get('value')) > np.mean(o2.get('value')) and np.mean(o1.get('value')) < 0:
        final = False
    elif o1.get('type') == o2.get('type') and o2.get('type')[0] == 'Cp' and np.mean(o1.get('value')) > np.mean(o2.get('value')) and dif1(np.mean(o1.get('value')),np.mean(o2.get('value'))) > delta and dif2(np.mean(o1.get('value')),np.mean(o2.get('value'))) >= epsilon:
        final = False
    else:
        final = True
    return final

In [19]:
def MR_mirror(o1,o2,delta,epsilon):
    final = bool()
    if not( all(elemento == o1.get('type')[0] for elemento in o1.get('type')) or all(elemento == o2.get('type')[0] for elemento in o2.get('type'))):
        final = False
    elif o1.get('type')!= o2.get('type'):
        final = False
    elif o1.get('type') == o2.get('type') and o1.get('type')[0] == 'Mate' and np.mean(o1.get('value')) != (-1 * np.mean(o2.get('value'))):
        final = False
    elif o1.get('type') == o2.get('type') and o1.get('type')[0] == 'Cp' and o1.get('type')[0] == 'Cp' and dif1(np.mean(o1.get('value')),(-1 * np.mean(o2.get('value')))) > delta and dif2(np.mean(o1.get('value')),(-1*np.mean(o2.get('value')))) >= epsilon:
        final = False
    else:
        final = True
    return final

### Main function MRequi, sim_axis

In [20]:
def mainEqui(dataset, time, rep, delta, epsilon):
    pos1 = []
    pos2 = []
    results1 = []
    results2 = []
    MR = []
    index = []
    for i in range(len(dataset)):
        axis = sim_axis(dataset[i])
        if bothcheck(dataset[i]) == False and bothcheck(axis) == False and checkMover(dataset[i]) == False:
            if i%5==0: 
                print(i)
            try:
                ev_source = evaluationND(dataset[i],time,rep)
                ev_followup = evaluationND(axis, time,rep)
                MR.append(MR_equi(ev_source, ev_followup, delta, epsilon)) ## adding MR to dataframe
                pos1.append(dataset[i]) ## adding original position to dataframe
                pos2.append(axis) ## adding transformed position to dataframe
                results1.append(ev_source) ## adding evaluation of original position to dataframe
                results2.append(ev_followup) ## adding evaluation of transformed position to dataframe
                index.append(i) ## adding index to dataframe
            except:
                    pass
    d = {'index' : index, 'pos1' : pos1, 'evaluation pos1' : results1, 'pos2' : pos2, 'evaluation pos2' : results2,
        'MR' : MR}
    df = pd.DataFrame(data=d) ## creating dataframe
    return df

An example with the first 40 positions of our dataset. 

In [ ]:
a = mainEqui(dataset[0:10], 3, 1, 0.25, 25)
print(a)

### Main function MRmirror

In [21]:
def mainMirr(dataset, time, rep, delta, epsilon):
    pos1 = []
    pos2 = []
    results1 = []
    results2 = []
    MR = []
    index = []
    for i in range(len(dataset)):
        mirror = sim_mirror(dataset[i])
        if bothcheck(dataset[i]) == False and bothcheck(mirror) == False and checkMover(dataset[i]) == False:
            if i%5==0: 
                print(i)
            try:
                ev_source = evaluationND(dataset[i],time, rep)
                ev_followup = evaluationND(mirror, time, rep)
                MR.append(MR_mirror(ev_source, ev_followup, delta, epsilon)) ## adding MR to dataframe
                pos1.append(dataset[i]) ## adding original position to dataframe
                pos2.append(mirror) ## adding transformed position to dataframe
                results1.append(ev_source) ## adding evaluation of original position to dataframe
                results2.append(ev_followup) ## adding evaluation of transformed position to dataframe
                index.append(i) ## adding index to dataframe
            except:
                    pass
    d = {'index' : index, 'pos1' : pos1, 'evaluation pos1' : results1, 'pos2' : pos2, 'evaluation pos2' : results2,
        'MR' : MR}
    df = pd.DataFrame(data=d) ## creating dataframe
    return df

An example with the first 10 positions of our dataset. 

In [22]:
a = mainMirr(dataset[0:10], 3, 1, 0.25, 25)
print(a)

0
5
   index                                            pos1  \
0      0            1b1B1Qr1/7p/6r1/2P5/4Rk2/1K6/4B3/8 b   
1      1                1B1B2K1/1B6/5N2/6k1/8/8/8/4nq2 b   
2      2  1b1b2K1/1r6/2P4p/1b1p4/4N1k1/3B2Q1/2q5/2N2q2 b   
3      5       1b1b4/1N5r/1P1p3B/8/3KP3/3P2k1/3PB3/1N6 b   
4      7            1b1B4/1P6/2B5/3r2kN/2r4P/8/5K2/3B4 b   
5      8               1B1b4/1Q2b3/8/6rn/3NK3/7k/2r5/8 b   
6      9                1b1B4/2b3PK/7P/8/1BP4k/4P3/8/8 b   

                       evaluation pos1  \
0    {'type': ['Cp'], 'value': [1053]}   
1    {'type': ['Cp'], 'value': [-101]}   
2  {'type': ['Cp'], 'value': [-32749]}   
3     {'type': ['Cp'], 'value': [-53]}   
4     {'type': ['Cp'], 'value': [909]}   
5       {'type': ['Cp'], 'value': [9]}   
6    {'type': ['Cp'], 'value': [2081]}   

                                             pos2  \
0            8/4b3/1k6/4rK2/2p5/6R1/7P/1B1b1qR1 w   
1                4NQ2/8/8/8/6K1/5n2/1b6/1b1b2k1 w   
2  2n2Q2/2Q5/3b2q1/

### Main function MRequi, diag

In [24]:
def mainDiag(dataset, time, rep, delta, epsilon):
    pos1 = []
    pos2 = []
    results1 = []
    results2 = []
    MR = []
    index = []
    for i in range(len(dataset)):
        diag = sim_diag(dataset[i])
        if (not has_pawns(dataset[i])) and bothcheck(dataset[i]) == False and bothcheck(diag) == False and checkMover(dataset[i]) == False:
            if i%5==0: 
                print(i)
            try:
                ev_source = evaluationND(dataset[i],time, rep)
                ev_followup = evaluationND(diag, time, rep)
                MR.append(MR_equi(ev_source, ev_followup, delta, epsilon)) ## adding MR to dataframe
                pos1.append(dataset[i]) ## adding original position to dataframe
                pos2.append(diag) ## adding transformed position to dataframe
                results1.append(ev_source) ## adding evaluation of original position to dataframe
                results2.append(ev_followup) ## adding evaluation of transformed position to dataframe
                index.append(i) ## adding index to dataframe
            except:
                    pass
    d = {'index' : index, 'pos1' : pos1, 'evaluation pos1' : results1, 'pos2' : pos2, 'evaluation pos2' : results2,
        'MR' : MR}
    df = pd.DataFrame(data=d) ## creating dataframe
    return df

An example with the first 10 positions of our dataset. 

In [25]:
a = mainDiag(dataset[0:10],3, 10, 0.25, 25)
print(a)

   index                               pos1  \
0      1   1B1B2K1/1B6/5N2/6k1/8/8/8/4nq2 b   
1      8  1B1b4/1Q2b3/8/6rn/3NK3/7k/2r5/8 b   

                                     evaluation pos1  \
0  {'type': ['Cp', 'Cp', 'Cp', 'Cp', 'Cp', 'Cp', ...   
1  {'type': ['Cp', 'Cp', 'Cp', 'Cp', 'Cp', 'Cp', ...   

                                 pos2  \
0         8/4k2K/q4N2/n7/7B/8/6BB/8 b   
1  2k1n3/4r3/8/3K2b1/3N3b/1r6/6QB/8 b   

                                     evaluation pos2    MR  
0  {'type': ['Cp', 'Cp', 'Cp', 'Cp', 'Cp', 'Cp', ...  True  
1  {'type': ['Cp', 'Cp', 'Cp', 'Cp', 'Cp', 'Cp', ...  True  


### Main function MRbetter

In [27]:
def mainBet(dataset, time, rep, delta, epsilon):
    pos1 = []
    pos2 = []
    results1 = []
    results2 = []
    MR = []
    index = []
    for i in range(len(dataset)):
        replaced = replace(dataset[i])
        if bothcheck(dataset[i]) == False and bothcheck(replaced) == False and checkMover(dataset[i]) == False:
            if i%5==0: 
                print(i)
            try:
                ev_source = evaluationND(dataset[i],time, rep)
                ev_followup = evaluationND(replaced, time, rep)
                MR.append(MR_better(ev_source, ev_followup, delta, epsilon)) ## adding MR to dataframe
                pos1.append(dataset[i]) ## adding original position to dataframe
                pos2.append(replaced) ## adding transformed position to dataframe
                results1.append(ev_source) ## adding evaluation of original position to dataframe
                results2.append(ev_followup) ## adding evaluation of transformed position to dataframe
                index.append(i) ## adding index to dataframe
            except:
                    pass
    d = {'index' : index, 'pos1' : pos1, 'evaluation pos1' : results1, 'pos2' : pos2, 'evaluation pos2' : results2,
        'MR' : MR}
    df = pd.DataFrame(data=d) ## creating dataframe
    return df

An example with the first 10 positions of our dataset. 

In [ ]:
a = mainBet(dataset[0:10],3, 1, 0.25, 25)
print(a)

### Main function MRfirst

In [ ]:
def MRfirst(dataset, time, rep, delta, epsilon):
    pos1 = []
    pos2 = []
    results1 = []
    results2 = []
    MR = []
    index = []
    for i in range(len(dataset)):
        try:
        
            
            position = next_board(dataset[i], next_move(dataset[i],time))
            if bothcheck(dataset[i]) == False and checkMover(dataset[i]) == False:
                if i%5==0: 
                    print(i)
                try:
                    ev_source = evaluationND(dataset[i],time,rep)
                    ev_followup = evaluationND(position, time,rep)
                    MR.append(MR_equi(ev_source, ev_followup, delta, epsilon)) ## adding MR to dataframe
                    pos1.append(dataset[i]) ## adding original position to dataframe
                    pos2.append(position) ## adding transformed position to dataframe
                    results1.append(ev_source) ## adding evaluation of original position to dataframe
                    results2.append(ev_followup) ## adding evaluation of transformed position to dataframe
                    index.append(i) ## adding index to dataframe
                except:
                        pass
        except:
            pass
    d = {'index' : index, 'pos1' : pos1, 'evaluation pos1' : results1, 'pos2' : pos2, 'evaluation pos2' : results2,
        'MR' : MR}
    df = pd.DataFrame(data=d) ## creating dataframe
    return df

An example with the first 10 positions of our dataset. 

In [ ]:
a = mainFirst(dataset[0:10],3, 1, 0.25, 25)
print(a)